# Reference

- https://learn.microsoft.com/en-us/rest/api/cost-management/retail-prices/azure-retail-prices
- https://azureprice.net
- https://azure.microsoft.com/en-in/pricing/details/virtual-machines/linux/
- https://github.com/doitintl/azure-instances.info

In [1]:
import requests
import pandas as pd

In [2]:

azure_vm_url = 'https://azure.microsoft.com/api/v3/pricing/virtual-machines/page/details/linux/?culture=en-in&showLowPriorityOffers=false'

In [3]:

response = requests.get(azure_vm_url)

In [4]:

response

<Response [200]>

In [5]:
response_json = response.json()

In [6]:
vm_info = response_json['attributesByOffer']

In [7]:
vm_info_df = pd.DataFrame(vm_info.values())

In [8]:
vm_info_df.head()

,series,nameLocKey,instanceName,cores,category,ram,tier,hasPayGo,isInPreview,diskSize,...,hasSpot,isVcpu,isConstrainedCore,isBaseVm,activeCores,productId,nvMeDisk,gpu,fpga,pmem
0,av2,a1v2,A1 v2,1,generalpurpose,2.0,standard,True,False,10.0,...,True,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN
1,av2,a2mv2,A2m v2,2,generalpurpose,16.0,standard,True,False,20.0,...,True,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN
2,av2,a2v2,A2 v2,2,generalpurpose,4.0,standard,True,False,20.0,...,True,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN
3,av2,a4mv2,A4m v2,4,generalpurpose,32.0,standard,True,False,40.0,...,True,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN
4,av2,a4v2,A4 v2,4,generalpurpose,8.0,standard,True,False,40.0,...,True,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
vm_info_df.columns

Index(['series', 'nameLocKey', 'instanceName', 'cores', 'category', 'ram',
       'tier', 'hasPayGo', 'isInPreview', 'diskSize', 'type', 'hasSpot',
       'isVcpu', 'isConstrainedCore', 'isBaseVm', 'activeCores', 'productId',
       'nvMeDisk', 'gpu', 'fpga', 'pmem'],
      dtype='object')

In [10]:
vm_info_df.to_csv("data/azure_vm_info.csv")

In [11]:
pricing_url = "https://prices.azure.com/api/retail/prices?$filter=serviceName eq 'Virtual Machines' and armRegionName eq 'southcentralus' and priceType eq 'Consumption'"

response = requests.get(pricing_url).json()
items = response['Items']
next_link =  response.get('NextPageLink')
while(next_link):
	response = requests.get(next_link)
	response_json = response.json()
	if response_json.get('Items'):
		items.extend(response_json.get('Items'))
	next_link =  response_json.get('NextPageLink')

In [12]:
pricing_df = pd.DataFrame(items)

In [13]:
pricing_df.head()

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,productId,...,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,effectiveEndDate
0,USD,0.0,4.838000,4.838000,southcentralus,US South Central,2020-08-01T00:00:00Z,0016083a-928f-56fd-8eeb-39287dcf676d,E64 v4,DZH318Z0D1L7,...,Virtual Machines Ev4 Series,E64 v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_E64_v4,NaN
1,USD,0.0,0.434000,0.434000,southcentralus,US South Central,2022-06-01T00:00:00Z,0050211e-db45-538f-b411-4c48804aa0ca,DC8ds_v3 Spot,DZH318Z08NRF,...,DCdsv3 Series Linux,standard_DC8ds_v3 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_DC8ds_v3,NaN
2,USD,0.0,1.136479,1.136479,southcentralus,US South Central,2023-02-01T00:00:00Z,005bbecd-034c-5d82-b927-1656f77dcb13,D48s v3 Spot,DZH318Z0BPWD,...,Virtual Machines DSv3 Series Windows,D48s v3 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D48s_v3,2023-02-28T23:59:00Z
3,USD,0.0,0.099000,0.099000,southcentralus,US South Central,2017-05-10T00:00:00Z,00888607-2147-4a79-849d-55bbd09dc37d,A8m v2 Low Priority,DZH318Z0BPRJ,...,Virtual Machines Av2 Series,A8m v2 Low Priority,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A8m_v2,NaN
4,USD,0.0,0.747906,0.747906,southcentralus,US South Central,2023-02-01T00:00:00Z,00bdbef3-8af7-58af-a4c4-61e1f0500039,D32s v4 Spot,DZH318Z0CP07,...,Virtual Machines Dsv4 Series Windows,D32s v4 Spot,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D32s_v4,2023-02-28T23:59:00Z


In [14]:
pricing_df.columns

Index(['currencyCode', 'tierMinimumUnits', 'retailPrice', 'unitPrice',
       'armRegionName', 'location', 'effectiveStartDate', 'meterId',
       'meterName', 'productId', 'skuId', 'availabilityId', 'productName',
       'skuName', 'serviceName', 'serviceId', 'serviceFamily', 'unitOfMeasure',
       'type', 'isPrimaryMeterRegion', 'armSkuName', 'effectiveEndDate'],
      dtype='object')

In [15]:
pricing_df.rename({'meterName': 'instanceName'}, axis='columns', inplace=True)

In [16]:
len(pricing_df)

3935

In [17]:
pricing_df = pricing_df[~pricing_df['instanceName'].str.contains('Spot')] 
pricing_df = pricing_df[~pricing_df['instanceName'].str.contains('Priority')]

In [18]:
len(pricing_df)

1381

In [19]:
pricing_df.to_csv("data/azure_vm_pricing.csv")

In [20]:
final_df = pd.merge(vm_info_df, pricing_df, on='instanceName')

In [21]:
final_df.head()

,series,nameLocKey,instanceName,cores,category,ram,tier,hasPayGo,isInPreview,diskSize,...,productName,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type_y,isPrimaryMeterRegion,armSkuName,effectiveEndDate
0,av2,a1v2,A1 v2,1,generalpurpose,2.0,standard,True,False,10.0,...,Virtual Machines Av2 Series,A1 v2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A1_v2,NaN
1,av2,a1v2,A1 v2,1,generalpurpose,2.0,standard,True,False,10.0,...,Virtual Machines Av2 Series Windows,A1 v2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A1_v2,NaN
2,av2,a2mv2,A2m v2,2,generalpurpose,16.0,standard,True,False,20.0,...,Virtual Machines Av2 Series Windows,A2m v2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A2m_v2,NaN
3,av2,a2mv2,A2m v2,2,generalpurpose,16.0,standard,True,False,20.0,...,Virtual Machines Av2 Series,A2m v2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A2m_v2,NaN
4,av2,a2v2,A2 v2,2,generalpurpose,4.0,standard,True,False,20.0,...,Virtual Machines Av2 Series,A2 v2,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_A2_v2,NaN


In [22]:
final_df.columns

Index(['series', 'nameLocKey', 'instanceName', 'cores', 'category', 'ram',
       'tier', 'hasPayGo', 'isInPreview', 'diskSize', 'type_x', 'hasSpot',
       'isVcpu', 'isConstrainedCore', 'isBaseVm', 'activeCores', 'productId_x',
       'nvMeDisk', 'gpu', 'fpga', 'pmem', 'currencyCode', 'tierMinimumUnits',
       'retailPrice', 'unitPrice', 'armRegionName', 'location',
       'effectiveStartDate', 'meterId', 'productId_y', 'skuId',
       'availabilityId', 'productName', 'skuName', 'serviceName', 'serviceId',
       'serviceFamily', 'unitOfMeasure', 'type_y', 'isPrimaryMeterRegion',
       'armSkuName', 'effectiveEndDate'],
      dtype='object')

In [23]:
final_df.to_csv("data/azure_vm.csv")

In [24]:
final_df[['cores', 'ram', 'unitPrice']].describe()

,cores,ram,unitPrice
count,952.000000,952.000000,952.000000
mean,34.144958,314.818803,4.390882
std,41.495822,765.286474,8.767047
min,1.000000,0.500000,0.006240
25%,8.000000,32.000000,0.629000
50%,16.000000,128.000000,2.170000
75%,48.000000,384.000000,5.096000
max,416.000000,11400.000000,138.120000
